In [16]:
import sys  
import os
sys.path.insert(0, '..')
from AT_funs_appendix import *
from AT_funs import read_xlsx

In [17]:
filename_bm = 'FTSEMIB.MI.xlsx'
bm_df = read_xlsx(filename_bm)
bm_name = filename_bm.replace('xlsx', '')
bm_df['date'] = pd.to_datetime(bm_df['date'])
bm_df['USD'] = 1

In [18]:
params = ['2014-12-31', None, 63, 0.05, 0.05, 1.5, 2,5,2.5,3]
start,end,vlty_n,dist_pct,retrace_pct,threshold,dgt,d_vol,r_vol,lvl= [params[h] for h in range(len(params))]

rel_var = ['FTSEMIB.MI','close', 'USD']
bm_ticker, bm_col, ccy_col = [rel_var[h] for h in range(len(rel_var))]
# lvl

In [19]:
window = 100
st= fast = 50
lt = slow = 200

batch_size = 1
show_batch = True
save_ticker_df = False
save_last_row_df = False
save_regime_df = False

In [20]:
# website = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
web_df_cols = ['Symbol','Security','GICS Sector','GICS Sub-Industry']
regime_cols = ['rg','rrg',
    'smac'+str(st)+str(lt),'smar'+str(st)+str(lt), 'bohl'+str(slow),
    'borr'+str(slow),'tth'+str(fast)+'l'+str(slow),'ttr'+str(fast)+'r'+str(slow)]

swings_cols = ['flr_dt','flr','clg_dt', 'clg', 'rg_ch', 
    'hi'+str(lvl)+'_dt','hi'+str(lvl),'lo'+str(lvl)+'_dt','lo'+str(lvl) ,
      'rflr_dt', 'rflr', 'rclg_dt', 'rclg', 'rrg_ch',
    'rh'+str(lvl)+'_dt','rh'+str(lvl),'rl'+str(lvl)+'_dt','rl'+str(lvl) ]
symbol_cols = ['ticker','date','close']

last_row_df_cols = symbol_cols+['score']+regime_cols+swings_cols

In [21]:
folder_path = os.getcwd()  
  
# Get all xlsx files in folder (except ftsemib.xlsx)  
tickers_list = [file for file in os.listdir(folder_path) if file.endswith(".xlsx") and file != "FTSEMIB.MI.xlsx"]  
# tickers_list = tickers_list[:20] 
# tickers_list

In [22]:
# Read all xlsx files  
dfs = []
    
for file in tickers_list:      
    file_path = os.path.join(folder_path, file)      
    df = pd.read_excel(file_path)  
    if df.shape[0] >= 1000:  
        dfs.append(df)  

In [23]:
regime_df = pd.DataFrame()
last_row_df = pd.DataFrame()
dfsr = []
last_row_list = []
failed = []

In [24]:
loop_size = len(tickers_list)+1

In [25]:
def last_row_dictionary(df, ticker):
    
    df_cols = list(df.columns)
    col_dict = {'Symbol':str.upper(ticker),'date':df['date'].max().strftime('%Y%m%d')}
    for i, col_name in enumerate(df_cols):
        if pd.isnull(df.iloc[-1,i]):
            try:
                last_index = df[pd.notnull(df.iloc[:,i])].index[-1]
                len_last_index = len(df[:last_index]) - 1
                col_dict.update({col_name + '_dt': last_index.strftime('%Y%m%d')})
                col_dict.update({col_name : df.iloc[len_last_index,i]})
            except:
                col_dict.update({col_name + '_dt':np.nan})
                col_dict.update({col_name : np.nan})
        else:
            col_dict.update({col_name : df.iloc[-1,i]})
    return col_dict

In [26]:

for t in range(1,loop_size): 
    m = (t - 1) * batch_size
    n = t * batch_size
    batch_list = tickers_list[m:n]
    if show_batch:
        print(batch_list,m,n)
        
    try:
        batch_download = dfs[t-1]
        
        for flat, ticker in enumerate(batch_list):
            # df = yf_droplevel(batch_download,ticker)           
            df = swings2(batch_download,lvl,vlty_n,dgt,d_vol,dist_pct,r_vol,retrace_pct,bm_df,bm_col,rel = False)
            df = regime(df,vlty_n,threshold,lvl=2,rel = False)
            df = swings2(batch_download,lvl,vlty_n,dgt,d_vol,dist_pct,r_vol,retrace_pct,bm_df,bm_col,rel = True)
            df = regime(df,vlty_n,threshold,lvl=2,rel = True)          
            _o,_h,_l,_c = lower_upper_OHLC(df,relative = False)
            df['sma'+str(_c)[:1]+str(st)+str(lt)] = regime_sma(df,_c,st,lt)
            df['bo'+str(_h)[:1]+str(_l)[:1]+ str(slow)] = regime_breakout(df,_h,_l,window)
            df['tt'+str(_h)[:1]+str(fast)+str(_l)[:1]+ str(slow)] = turtle_trader(df, _h, _l, slow, fast)
            _o,_h,_l,_c = lower_upper_OHLC(df,relative = True) 
            df['sma'+str(_c)[:1]+str(st)+str(lt)] = regime_sma(df,_c,st,lt)
            df['bo'+str(_h)[:1]+str(_l)[:1]+ str(slow)] = regime_breakout(df,_h,_l,window)
            df['tt'+str(_h)[:1]+str(fast)+str(_l)[:1]+ str(slow)] = turtle_trader(df, _h, _l, slow, fast)
                
        try:
            last_row_list.append(last_row_dictionary(df, ticker))
        except:
            failed.append(ticker) 
    except:
        failed.append(ticker)
last_row_df = pd.DataFrame.from_dict(last_row_list)

if save_last_row_df:
    last_row_df.to_csv('last_row_df_'+ str(last_row_df['date'].max())+'.csv', date_format='%Y%m%d')
print('failed',failed)



['RLT.MI.xlsx'] 0 1
['AIM.MI.xlsx'] 1 2
['ABP.MI.xlsx'] 2 3
['MAPS.MI.xlsx'] 3 4
['GE.MI.xlsx'] 4 5
['EVISO.MI.xlsx'] 5 6
['AVIO.MI.xlsx'] 6 7
['GM.MI.xlsx'] 7 8
['RAT.MI.xlsx'] 8 9
['CAI.MI.xlsx'] 9 10
['GHC.MI.xlsx'] 10 11
['AEF.MI.xlsx'] 11 12


/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):


['ART.MI.xlsx'] 12 13
['TPRO.MI.xlsx'] 13 14
['ESPE.MI.xlsx'] 14 15
['EUK.MI.xlsx'] 15 16
['MTVFR.MI.xlsx'] 16 17
['CE.MI.xlsx'] 17 18
['FAE.MI.xlsx'] 18 19
['ACQ.MI.xlsx'] 19 20
['UNI.MI.xlsx'] 20 21
['CNS.MI.xlsx'] 21 22
['IMS.MI.xlsx'] 22 23
['BC.MI.xlsx'] 23 24
['OVS.MI.xlsx'] 24 25
['LMG.MI.xlsx'] 25 26
['GML.MI.xlsx'] 26 27
['ELES.MI.xlsx'] 27 28
['PTR.MI.xlsx'] 28 29
['PIA.MI.xlsx'] 29 30
['CDG.MI.xlsx'] 30 31
['LASIA.MI.xlsx'] 31 32
['UCG.MI.xlsx'] 32 33
['ILP.MI.xlsx'] 33 34
['AGAIN.MI.xlsx'] 34 35
['CELL.MI.xlsx'] 35 36
['TCM.MI.xlsx'] 36 37
['TPS.MI.xlsx'] 37 38
['IMP.MI.xlsx'] 38 39
['ESIGM.MI.xlsx'] 39 40
['ESAU.MI.xlsx'] 40 41
['SEIF.MI.xlsx'] 41 42
['TFIN.MI.xlsx'] 42 43


/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):
/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):


['VLS.MI.xlsx'] 43 44
['SCM.MI.xlsx'] 44 45
['BST.MI.xlsx'] 45 46
['GEL.MI.xlsx'] 46 47
['DAL.MI.xlsx'] 47 48
['ILTY.MI.xlsx'] 48 49
['AUTME.MI.xlsx'] 49 50
['IOT.MI.xlsx'] 50 51
['FF.MI.xlsx'] 51 52
['AC5.MI.xlsx'] 52 53
['KK.MI.xlsx'] 53 54
['FBK.MI.xlsx'] 54 55
['LS.MI.xlsx'] 55 56
['TLS.MI.xlsx'] 56 57


...

['RAD.MI.xlsx'] 236 237
['AZM.MI.xlsx'] 237 238
['MARR.MI.xlsx'] 238 239
['NVP.MI.xlsx'] 239 240
['ABC.MI.xlsx'] 240 241
['ECK.MI.xlsx'] 241 242
['AGP.MI.xlsx'] 242 243
['MNL.MI.xlsx'] 243 244
['OJM.MI.xlsx'] 244 245
['DIT.MI.xlsx'] 245 246
['REY.MI.xlsx'] 246 247
['ASC.MI.xlsx'] 247 248
['IIG.MI.xlsx'] 248 249
['CY4.MI.xlsx'] 249 250
['TEN.MI.xlsx'] 250 251
['MOL.MI.xlsx'] 251 252
['CBB.MI.xlsx'] 252 253
['MGS.MI.xlsx'] 253 254
['ERFO.MI.xlsx'] 254 255
['GVS.MI.xlsx'] 255 256
['NRST.MI.xlsx'] 256 257
['ZV.MI.xlsx'] 257 258
['ABT.MI.xlsx'] 258 259
['CMB.MI.xlsx'] 259 260
['XHS.MI.xlsx'] 260 261
['SBC.MI.xlsx'] 261 262
['MAIRE.MI.xlsx'] 262 263
['GEO.MI.xlsx'] 263 264
['FPE.MI.xlsx'] 264 265
['A2A.MI.xlsx'] 265 266
['FOS.MI.xlsx'] 266 267
['ENI.MI.xlsx'] 267 268
['WBS.MI.xlsx'] 268 269
['RDF.MI.xlsx'] 269 270


/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):
/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):


['TRN.MI.xlsx'] 270 271
['BGN.MI.xlsx'] 271 272


/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):


['JUVE.MI.xlsx'] 272 273
['ARN.MI.xlsx'] 273 274
['ANIM.MI.xlsx'] 274 275
['CML.MI.xlsx'] 275 276
['HER.MI.xlsx'] 276 277
['PHN.MI.xlsx'] 277 278
['TIT.MI.xlsx'] 278 279
['S24.MI.xlsx'] 279 280
['COM.MI.xlsx'] 280 281
['SMN.MI.xlsx'] 281 282
['EXAI.MI.xlsx'] 282 283
['ALA.MI.xlsx'] 283 284
['INW.MI.xlsx'] 284 285
['U24.MI.xlsx'] 285 286
['LUVE.MI.xlsx'] 286 287
['IEG.MI.xlsx'] 287 288
['D.MI.xlsx'] 288 289
['CIRC.MI.xlsx'] 289 290
['VNT.MI.xlsx'] 290 291
['MONC.MI.xlsx'] 291 292
['GO.MI.xlsx'] 292 293
['CYB.MI.xlsx'] 293 294
['TIME.MI.xlsx'] 294 295
['GPI.MI.xlsx'] 295 296
['US.MI.xlsx'] 296 297
['TYA.MI.xlsx'] 297 298
['SSL.MI.xlsx'] 298 299
['TGYM.MI.xlsx'] 299 300
['DNR.MI.xlsx'] 300 301
['PMT.MI.xlsx'] 301 302
['BELL.MI.xlsx'] 302 303
['B.MI.xlsx'] 303 304
['SAB.MI.xlsx'] 304 305
['DOV.MI.xlsx'] 305 306
['POPR.MI.xlsx'] 306 307
['GRN.MI.xlsx'] 307 308
['SGF.MI.xlsx'] 308 309
['CFV.MI.xlsx'] 309 310
['GSR.MI.xlsx'] 310 311
['IG.MI.xlsx'] 311 312
['ALFO.MI.xlsx'] 312 313
['FUM.MI.xls

In [27]:
last_row_df['score']= last_row_df[regime_cols].sum(axis=1)
sectors = pd.read_excel('sectors.xlsx')

In [28]:
regime_df = sectors.set_index('ticker').join(
    last_row_df[last_row_df_cols].set_index('ticker'), how='inner').sort_values(by='score')

if save_regime_df:
    regime_df.to_csv('regime_df_'+ str(last_row_df['date'].max())+'.csv', date_format='%Y%m%d')

In [33]:
groupby_cols = ['score'] + regime_cols
sort_key = ['name', 'sector']
regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
    by= 'score').style.background_gradient(
    subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

,,score,rg,rrg,smac50200,smar50200,bohl200,borr200,tth50l200,ttr50r200
name,sector,,,,,,,,,
JUVENTUS FC,Consumer Services,-8,-1,-1,-1,-1,-1,-1,-1,-1
ZIGNAGO VETRO,Process Industries,-8,-1,-1,-1,-1,-1,-1,-1,-1
DOXEE,Technology Services,-8,-1,-1,-1,-1,-1,-1,-1,-1
DOVALUE,Finance,-8,-1,-1,-1,-1,-1,-1,-1,-1
DIGITOUCH,Commercial Services,-8,-1,-1,-1,-1,-1,-1,-1,-1
DATALOGIC,Electronic Technology,-8,-1,-1,-1,-1,-1,-1,-1,-1
CULTI MILANO,Consumer Non-Durables,-8,-1,-1,-1,-1,-1,-1,-1,-1
CSP INT IND CALZE,Consumer Non-Durables,-8,-1,-1,-1,-1,-1,-1,-1,-1
LANDI RENZO,Producer Manufacturing,-8,-1,-1,-1,-1,-1,-1,-1,-1


In [32]:
groupby_cols = ['score'] + regime_cols
sort_key = ['sector']
regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
    by= 'score').style.background_gradient(
    subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

pd.set_option('display.float_format', lambda x: '%.2f' % x)
groupby_cols = ['score'] + regime_cols
sort_key = ['sector']
regime_df.groupby(sort_key)[groupby_cols].sum().sort_values(
    by= 'score').style.background_gradient(
    subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

,score,rg,rrg,smac50200,smar50200,bohl200,borr200,tth50l200,ttr50r200
sector,,,,,,,,,
Consumer Durables,-8e+01,-6,-2e+01,-3,-2e+01,-1,-2e+01,-5,-2e+01
Technology Services,-7e+01,-3,-1e+01,-3,-2e+01,-1,-2e+01,-5,-1e+01
Consumer Non-Durables,-7e+01,-8,-1e+01,-4,-1e+01,-4,-1e+01,-6,-1e+01
Utilities,-5e+01,2,-6,-4,-1e+01,0,-1e+01,1,-1e+01
Consumer Services,-4e+01,0,-1e+01,2,-2e+01,2,-1e+01,2,-8
Communications,-4e+01,-5,-4,-5,-5,-5,-5,-6,-7
Producer Manufacturing,-4e+01,9,-2e+01,2,-2e+01,6,-1e+01,4,-1e+01
Distribution Services,-2e+01,0,-6,-2,-6,0,-4,2,-3
Health Services,-1e+01,1,-3,-1,-3,-1,-3,0,-3


In [ ]:
# groupby_cols = ['score'] + regime_cols
# sort_key = ['GICS Sub-Industry']
# regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
#     by= 'score').style.background_gradient(
#     subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

In [ ]:
# groupby_cols = ['score'] + regime_cols
# sort_key = ['GICS Sector','GICS Sub-Industry']
# regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
#     by= ['GICS Sector','score']).style.background_gradient(
#     subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')